In [1]:
import torch
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', pretrained=True)
# or any of these variants
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet34', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet50', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet101', pretrained=True)
# model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet152', pretrained=True)
model.eval()

Using cache found in C:\Users\Matt/.cache\torch\hub\pytorch_vision_v0.6.0


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

We do some simple logic flow to handle either png or jpg images. The pre-trained torch models require images to be normalized into mini-batches of 3-channel RGB images of shape `(3xHxW)` where `H` and `W` are atleast `224`. The images have to be loaded in to a range of `[0, 1]` and then normalized using `mean = [0.485, 0.456, 0.406]` and `std = [0.229, 0.224, 0.225]`.

In [8]:
# Download an example image from the pytorch website
import urllib
from PIL import Image
from torchvision import transforms

url, filename = ("https://user-images.githubusercontent.com/15249120/107858311-9b08cd00-6e01-11eb-9891-6caaac901de7.png", "./img/hp.png")
#url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)
input_image = Image.open(filename)
if input_image.mode != 'RGB':
    input_image = input_image.convert('RGB')
input_image.mode

'RGB'

In [11]:
# sample execution (requires torchvision)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)
print(probabilities)

e+00,  1.2605e+00,
         1.6055e+00,  4.0594e+00, -3.7257e+00,  1.9499e-01,  5.7948e+00,
        -4.5434e+00,  1.3774e+00,  1.3026e+00,  5.4394e-01, -1.8425e+00,
        -1.0543e+00,  2.0610e+00,  1.7900e+00,  3.3995e+00,  5.6849e+00,
         1.9958e+00,  4.9095e-01, -4.9116e-01,  1.9823e+00, -2.4414e+00,
        -1.4520e+00,  3.9616e+00, -4.4381e-01, -3.8214e-01, -3.3330e-01,
         5.3863e+00,  2.9159e-01, -5.9827e-01,  2.7048e+00, -9.1224e-01,
         3.0214e-01,  2.9702e+00,  4.3191e+00,  1.2361e+00,  1.5278e-01,
        -2.0109e+00, -1.2003e+00, -2.9844e+00,  3.4420e-01,  4.2787e+00,
         2.8461e+00,  2.6912e-01,  2.2900e+00,  2.0522e-01,  1.8382e+00,
         1.0867e+00,  3.7735e+00,  1.0517e+00,  3.4661e+00,  1.5830e+00,
        -3.7087e+00,  2.2527e+00,  8.2049e-02,  2.0804e-01, -3.5766e-01,
         3.3546e+00,  7.9393e-01,  6.3751e-01,  1.1036e+00, -1.7985e+00,
         2.1818e+00,  3.7322e+00, -3.4788e+00,  3.0517e+00, -1.4758e+00,
         5.6887e+00,  4.6413e+00

In [12]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt -OutFile imagenet_classes.txt

'wget' is not recognized as an internal or external command,
operable program or batch file.
